In [32]:
import os   
import pandas as pd

filename = os.path.join("data", "tableau_1.csv")

df = pd.read_csv(filename, sep=";")

/tmp/ipykernel_4738/2005949297.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep=";")


In [33]:
print(df.columns.tolist())


['Niveau', 'ZONE', 'ANNEE', 'PATHOLOGIE', 'SEXE', "Tranche d'age", 'ind_freq']


In [34]:
print(df.columns.tolist())


['Niveau', 'ZONE', 'ANNEE', 'PATHOLOGIE', 'SEXE', "Tranche d'age", 'ind_freq']


In [36]:
print(df.columns.tolist())
print(df.head())


['Niveau', 'ZONE', 'ANNEE', 'PATHOLOGIE', 'SEXE', "Tranche d'age", 'ind_freq']
         Niveau      ZONE  ANNEE                                   PATHOLOGIE  \
0  Départements  01 - Ain   2018  01000-Maladies infectieuses et parasitaires   
1  Départements  01 - Ain   2018  01000-Maladies infectieuses et parasitaires   
2  Départements  01 - Ain   2018  01000-Maladies infectieuses et parasitaires   
3  Départements  01 - Ain   2018  01000-Maladies infectieuses et parasitaires   
4  Départements  01 - Ain   2018  01000-Maladies infectieuses et parasitaires   

       SEXE     Tranche d'age ind_freq  
0     Homme  00-Moins d'un an      120  
1     Femme  00-Moins d'un an      103  
2  Ensemble  00-Moins d'un an      223  
3     Homme      01-1 à 4 ans      167  
4     Femme      01-1 à 4 ans      134  


In [37]:
# Extraction du code département avant le "-"
df["dept_code"] = df["ZONE"].str.extract(r"^(\d{2,3})")


In [38]:
print(df[["ZONE", "dept_code"]].head(10))


       ZONE dept_code
0  01 - Ain        01
1  01 - Ain        01
2  01 - Ain        01
3  01 - Ain        01
4  01 - Ain        01
5  01 - Ain        01
6  01 - Ain        01
7  01 - Ain        01
8  01 - Ain        01
9  01 - Ain        01


In [39]:
df = df[
    (df["Niveau"] == "Départements") &
    (df["ANNEE"] == 2022) &
    (df["SEXE"] == "Ensemble")
].copy()


In [40]:
print(df[["Niveau", "ANNEE", "SEXE"]].drop_duplicates())


               Niveau  ANNEE      SEXE
2610650  Départements   2022  Ensemble


In [41]:
# Split PATHOLOGIE sur le premier "-"
patho_split = df["PATHOLOGIE"].astype(str).str.split("-", n=1, expand=True)

df["PATHO_CODE"] = patho_split[0].str.strip()
df["PATHO_LIB"] = patho_split[1].str.strip()

# Contrôle visuel
print(df[["PATHOLOGIE", "PATHO_CODE", "PATHO_LIB"]].head(10))


                                          PATHOLOGIE PATHO_CODE  \
2610650  01000-Maladies infectieuses et parasitaires      01000   
2610653  01000-Maladies infectieuses et parasitaires      01000   
2610656  01000-Maladies infectieuses et parasitaires      01000   
2610659  01000-Maladies infectieuses et parasitaires      01000   
2610662  01000-Maladies infectieuses et parasitaires      01000   
2610665  01000-Maladies infectieuses et parasitaires      01000   
2610668  01000-Maladies infectieuses et parasitaires      01000   
2610671  01000-Maladies infectieuses et parasitaires      01000   
2610674  01000-Maladies infectieuses et parasitaires      01000   
2610677  01000-Maladies infectieuses et parasitaires      01000   

                                     PATHO_LIB  
2610650  Maladies infectieuses et parasitaires  
2610653  Maladies infectieuses et parasitaires  
2610656  Maladies infectieuses et parasitaires  
2610659  Maladies infectieuses et parasitaires  
2610662  Maladies

In [42]:
df["ind_freq"] = pd.to_numeric(df["ind_freq"], errors="coerce")


In [43]:
df["PATHOLOGIE"].drop_duplicates().head(20).tolist()


['01000-Maladies infectieuses et parasitaires',
 '01001-Maladies intestinales infectieuses',
 '01002-Tuberculose',
 '01003-Septicémies',
 '01004-Mal.sexuellement transmissibles',
 '01005-Hépatites virales',
 '01006-Maladies dues au V.I.H.',
 '02000-Tumeurs',
 '02007-Tumeurs malignes',
 '02008-T.M. lèvre _bouche_pharynx',
 "02009-Tumeur maligne de l'oesophage",
 "02010-Tumeur maligne de l'estomac",
 '02011-Tumeurs malignes colo-rectales',
 '02012-T.M.foie et voies biliaires intrahép',
 '02013-Tumeur maligne du pancréas',
 '02014-Tumeur maligne du larynx',
 '02015-T.M. trachée_bronches_poumon',
 '02016-Mélanome malin de la peau',
 '02017-Tumeur maligne du sein',
 "02018-Tumeur maligne de l'utérus"]

In [44]:
patho_to_spe = {
    # 01XXX → maladies infectieuses
    "01000": "Infectiologie",
    "01001": "Infectiologie",
    "01002": "Pneumologie",           # Tuberculose → classée en pneumo
    "01003": "Infectiologie",
    "01004": "Infectiologie",
    "01005": "Hépato-gastro-entérologie",
    "01006": "Infectiologie",

    # 02XXX → tumeurs (cancers)
    "02000": "Oncologie",
    "02007": "Oncologie",
    "02008": "Oncologie ORL",
    "02009": "Oncologie Digestive",
    "02010": "Oncologie Digestive",
    "02011": "Oncologie Digestive",
    "02012": "Oncologie Digestive",
    "02013": "Oncologie Digestive",
    "02014": "Oncologie ORL",
    "02015": "Oncologie Pneumologique",
    "02016": "Dermatologie",
    "02017": "Oncologie Sénologique",
    "02018": "Oncologie Gynécologique",
}


In [45]:
df["specialite"] = df["PATHO_CODE"].map(patho_to_spe).fillna("Autre")


In [46]:
print(df[["PATHO_CODE", "PATHO_LIB", "specialite"]].head(20))


        PATHO_CODE                              PATHO_LIB     specialite
2610650      01000  Maladies infectieuses et parasitaires  Infectiologie
2610653      01000  Maladies infectieuses et parasitaires  Infectiologie
2610656      01000  Maladies infectieuses et parasitaires  Infectiologie
2610659      01000  Maladies infectieuses et parasitaires  Infectiologie
2610662      01000  Maladies infectieuses et parasitaires  Infectiologie
2610665      01000  Maladies infectieuses et parasitaires  Infectiologie
2610668      01000  Maladies infectieuses et parasitaires  Infectiologie
2610671      01000  Maladies infectieuses et parasitaires  Infectiologie
2610674      01000  Maladies infectieuses et parasitaires  Infectiologie
2610677      01000  Maladies infectieuses et parasitaires  Infectiologie
2610680      01000  Maladies infectieuses et parasitaires  Infectiologie
2610683      01000  Maladies infectieuses et parasitaires  Infectiologie
2610686      01000  Maladies infectieuses et parasi

In [47]:
agg = (
    df.groupby(["dept_code", "specialite"], as_index=False)["ind_freq"]
      .sum()
      .rename(columns={"ind_freq": "charge_pathologique"})
)
print(agg.head())


  dept_code                 specialite  charge_pathologique
0        01                      Autre             878452.0
1        01               Dermatologie                220.0
2        01  Hépato-gastro-entérologie                 35.0
3        01              Infectiologie               6696.0
4        01                  Oncologie              37446.0


In [48]:
wide = agg.pivot_table(
    index="dept_code",
    columns="specialite",
    values="charge_pathologique",
    fill_value=0
).reset_index()

# Nettoyage esthétique
wide.columns.name = None

print(wide.head())


  dept_code     Autre  Dermatologie  Hépato-gastro-entérologie  Infectiologie  \
0        01  878452.0         220.0                       35.0         6696.0   
1        02  831779.0         214.0                       52.0         7585.0   
2        03  555980.0         228.0                       14.0         4600.0   
3        04  276648.0          40.0                       14.0         1386.0   
4        05  228814.0          60.0                        4.0         1813.0   

   Oncologie  Oncologie Digestive  Oncologie Gynécologique  Oncologie ORL  \
0    37446.0               2226.0                    244.0          469.0   
1    34508.0               2281.0                    237.0          521.0   
2    26628.0               1612.0                    207.0          317.0   
3    13667.0               1024.0                     94.0          158.0   
4    11200.0                628.0                     86.0          105.0   

   Oncologie Pneumologique  Oncologie Sénologique 

In [49]:
agg.to_csv("dept_specialite_long.csv", index=False)
wide.to_csv("dept_specialite_wide.csv", index=False)
